<a href="https://colab.research.google.com/github/Hushpuppyzac/DLI-Assignment/blob/main/DLI_Grp_Asgmt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
import requests
import nbformat
from IPython import get_ipython
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler

def run_notebook_from_github(url):
    print("🔄 Downloading notebook from GitHub...")
    try:
        response = requests.get(url)
        response.raise_for_status()
    except Exception as e:
        print(f"❌ Failed to download notebook: {e}")
        return None

    print("📖 Parsing notebook content...")
    try:
        notebook = nbformat.reads(response.text, as_version=4)
    except Exception as e:
        print(f"❌ Failed to parse notebook: {e}")
        return None

    ipython = get_ipython()
    if ipython is None:
        print("❌ This function must be run in a Jupyter environment.")
        return None

    print("⚙️ Running notebook cells...\n")
    # Execute code from the downloaded notebook directly in the current scope
    for i, cell in enumerate(notebook.cells):
        if cell.cell_type == 'code':
            try:
                print(f"▶️ Executing cell [{i + 1}]...")
                # Use exec to execute the code in the current global scope
                exec(cell.source, globals())
            except Exception as e:
                print(f"❌ Error in cell [{i + 1}]: {e}")
                # Continue execution even if a cell fails, to see all errors

    print("\n✅ All executable cells have been processed.")
    # Return the variables needed for the next steps
    return X_train_scaled, X_test_scaled, y_train_balanced, y_test

notebook_url = "https://raw.githubusercontent.com/Hushpuppyzac/DLI-Assignment/main/CleanedData.ipynb"
# Assign the returned variables to the global scope
notebook_results = run_notebook_from_github(notebook_url)

if notebook_results is not None:
    X_train_scaled, X_test_scaled, y_train_balanced, y_test = notebook_results
    # Ensure the target variables are integers after running the notebook code
    if isinstance(y_train_balanced, pd.Series):
        y_train_balanced = y_train_balanced.astype(int)
    if isinstance(y_test, pd.Series):
        y_test = y_test.astype(int)

🔄 Downloading notebook from GitHub...
📖 Parsing notebook content...
⚙️ Running notebook cells...

▶️ Executing cell [2]...

             INITIAL DATASET INFORMATION             
Total rows before any cleaning: 225,745
Initial Label distribution (raw data):


,count
Label,
DDoS,128027
BENIGN,97718



--- DataFrame Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225745 entries, 0 to 225744
Data columns (total 79 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0    Destination Port             225745 non-null  int64  
 1    Flow Duration                225745 non-null  int64  
 2    Total Fwd Packets            225745 non-null  int64  
 3    Total Backward Packets       225745 non-null  int64  
 4   Total Length of Fwd Packets   225745 non-null  int64  
 5    Total Length of Bwd Packets  225745 non-null  int64  
 6    Fwd Packet Length Max        225745 non-null  int64  
 7    Fwd Packet Length Min        225745 non-null  int64  
 8    Fwd Packet Length Mean       225745 non-null  float64
 9    Fwd Packet Length Std        225745 non-null  float64
 10  Bwd Packet Length Max         225745 non-null  int64  
 11   Bwd Packet Length Min        225745 non-null  int64  
 12   Bwd Packet Length M

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
count,225745.00000,2.257450e+05,225745.000000,225745.000000,225745.000000,2.257450e+05,225745.000000,225745.000000,225745.000000,225745.000000,...,225745.000000,225745.000000,2.257450e+05,2.257450e+05,2.257450e+05,2.257450e+05,2.257450e+05,2.257450e+05,2.257450e+05,2.257450e+05
mean,8879.61946,1.624165e+07,4.874916,4.572775,939.463346,5.960477e+03,538.535693,27.882221,164.826715,214.907242,...,3.311497,21.482753,1.848261e+05,1.293436e+04,2.080849e+05,1.776201e+05,1.032214e+07,3.611943e+06,1.287813e+07,7.755355e+06
std,19754.64740,3.152437e+07,15.422874,21.755356,3249.403484,3.921834e+04,1864.128991,163.324159,504.892965,797.411073,...,12.270018,4.166799,7.979250e+05,2.102737e+05,9.002350e+05,7.842602e+05,2.185303e+07,1.275689e+07,2.692126e+07,1.983109e+07
min,0.00000,-1.000000e+00,1.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,80.00000,7.118000e+04,2.000000,1.000000,26.000000,0.000000e+00,6.000000,0.000000,6.000000,0.000000,...,1.000000,20.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,80.00000,1.452333e+06,3.000000,4.000000,30.000000,1.640000e+02,20.000000,0.000000,8.666667,5.301991,...,2.000000,20.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,80.00000,8.805237e+06,5.000000,5.000000,63.000000,1.160100e+04,34.000000,6.000000,32.000000,10.263203,...,4.000000,20.000000,1.878000e+03,0.000000e+00,1.878000e+03,1.862000e+03,8.239725e+06,0.000000e+00,8.253838e+06,7.422849e+06
max,65532.00000,1.199999e+08,1932.000000,2942.000000,183012.000000,5.172346e+06,11680.000000,1472.000000,3867.000000,6692.644993,...,1931.000000,52.000000,1.000000e+08,3.950000e+07,1.000000e+08,1.000000e+08,1.200000e+08,6.530000e+07,1.200000e+08,1.200000e+08




--- DataFrame Columns after stripping whitespace and renaming ---
['Destination Port', 'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets', 'Total Length of Fwd Packets', 'Total Length of Bwd Packets', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean', 'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length', 'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s', 'Min Packet Length', 'Max Packet Length', 'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count', 'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', '

...


Final Columns (Testing Set after Feature Extraction):
['Destination Port', 'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets', 'Total Length of Fwd Packets', 'Total Length of Bwd Packets', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean', 'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Fwd Header Length', 'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s', 'Min Packet Length', 'Max Packet Length', 'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count', 'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count', 'URG Flag Count', 'ECE Flag Count', 'Down/Up Ra

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,pkt_length_diff,pkt_length_var_ratio,byte_ratio,duration_per_packet,avg_to_max_ratio
0,-0.496462,-0.511479,-0.245222,-0.168530,-0.303820,-0.138477,-0.292419,0.082431,-0.269685,-0.294430,...,-0.220676,-0.469879,-0.279191,-0.475267,-0.390354,-0.788134,-0.663447,-0.042185,-0.558311,0.632065
1,2.424285,-0.513378,0.004044,-0.123929,3.008621,-0.140547,5.431752,-0.182254,3.902286,5.708969,...,-0.220676,-0.469879,-0.279191,-0.475267,-0.390354,2.285842,1.203662,-0.042170,-0.578708,-0.982426
2,2.322853,-0.514890,-0.245222,-0.168530,-0.317288,-0.141196,-0.315716,-0.182254,-0.355898,-0.294430,...,-0.220676,-0.469879,-0.279191,-0.475267,-0.390354,-0.804659,-1.136167,-0.042185,-0.581654,-1.275411
3,2.147016,-0.221781,-0.245222,0.009873,-0.315569,-0.140448,-0.312742,-0.148465,-0.344892,-0.294430,...,-0.204030,-0.041240,-0.279191,-0.126852,0.081679,-0.804659,-0.842734,-0.042185,0.087018,1.115593
4,-0.495185,-0.219132,0.004044,-0.213131,-0.308692,-0.141196,-0.312742,-0.148465,-0.344892,-0.294430,...,-0.217260,-0.036873,-0.279191,-0.123302,0.086489,-0.804659,-0.842734,0.058275,0.228007,1.183205



Sample of Scaled Testing Data (First 5 Rows):


,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,pkt_length_diff,pkt_length_var_ratio,byte_ratio,duration_per_packet,avg_to_max_ratio
139654,2.095897,-0.360247,-0.245222,0.009873,-0.315569,-0.140448,-0.312742,-0.148465,-0.344892,-0.294430,...,-0.220676,-0.469879,-0.279191,-0.475267,-0.390354,-0.804659,-0.842734,-0.042185,-0.228870,1.115593
127815,-0.495185,-0.491359,-0.120589,0.009873,-0.309838,0.148192,-0.305803,-0.182254,-0.340001,-0.282545,...,-0.220676,-0.469879,-0.279191,-0.475267,-0.390354,0.751919,0.190441,-0.042185,-0.541403,-0.765248
111648,2.589378,-0.486148,-0.245222,0.054474,-0.315569,-0.140298,-0.312742,-0.148465,-0.344892,-0.294430,...,-0.220676,-0.469879,-0.279191,-0.475267,-0.390354,-0.804659,-0.842734,-0.042185,-0.525461,1.066421
76785,-0.495185,-0.512893,-0.120589,0.009873,-0.309838,0.148192,-0.305803,-0.182254,-0.340001,-0.282545,...,-0.220676,-0.469879,-0.279191,-0.475267,-0.390354,0.751919,0.190441,-0.042185,-0.578247,-0.765248
72328,-0.495185,-0.505281,0.004044,-0.213131,-0.308692,-0.141196,-0.312742,-0.148465,-0.344892,-0.294430,...,-0.220676,-0.469879,-0.279191,-0.475267,-0.390354,-0.804659,-0.842734,0.058275,-0.555356,1.183205




✅ All executable cells have been processed.


Isaac Khoo Rui Min (CNN)

In [41]:
# ✅ STEP 2: Train CNN Classifier on Cleaned Dataset
# This step assumes the preprocessing from the 'cleaned_data_corrected' Canvas
# has already been executed and its outputs (X_train_scaled, y_train_balanced, X_test_scaled, y_test)
# are available in the environment.

# 🔹 Import required libraries for model training and evaluation
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import numpy as np # Import numpy for reshaping

# 🔹 Step 2.1: Verify if preprocessed data exists
try:
    # These variables should be available after running the preprocessing steps
    X_train_scaled, y_train_balanced, X_test_scaled, y_test
except NameError:
    raise Exception("❌ Preprocessed data (X_train_scaled, y_train_balanced, X_test_scaled, y_test) not found. "
                    "Please ensure the 'cleaned_data_corrected' Canvas is executed first.")

# Convert y_train_balanced and y_test to numerical type (good as a safeguard)
y_train_balanced = y_train_balanced.astype(int)
y_test = y_test.astype(int)

# Convert DataFrames to NumPy arrays before reshaping
X_train_scaled_np = X_train_scaled.to_numpy()
X_test_scaled_np = X_test_scaled.to_numpy()


# 🔹 Step 2.2: Reshape data for CNN (Conv1D expects 3D input: samples, timesteps/features, channels)
# Here, each feature is treated as a timestep, and we have 1 channel.
X_train_cnn = X_train_scaled_np.reshape(X_train_scaled_np.shape[0], X_train_scaled_np.shape[1], 1)
X_test_cnn = X_test_scaled_np.reshape(X_test_scaled_np.shape[0], X_test_scaled_np.shape[1], 1)

print(f"\nReshaped X_train for CNN: {X_train_cnn.shape}")
print(f"Reshaped X_test for CNN: {X_test_cnn.shape}")


# 🔹 Step 2.3: Build the CNN model
cnn_model = models.Sequential([
    # First Convolutional Block
    layers.Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(X_train_cnn.shape[1], 1)),
    layers.MaxPooling1D(pool_size=2),
    layers.Dropout(0.25), # Added dropout for regularization

    # Second Convolutional Block
    layers.Conv1D(filters=64, kernel_size=3, activation='relu'),
    layers.MaxPooling1D(pool_size=2),
    layers.Dropout(0.25), # Added dropout

    # Flatten the output for the Dense layers
    layers.Flatten(),

    # Dense layers for classification
    layers.Dense(units=128, activation='relu'),
    layers.Dropout(0.5), # Added dropout

    # Output layer for binary classification (DDoS or BENIGN)
    layers.Dense(units=1, activation='sigmoid') # Sigmoid for binary classification
])

# 🔹 Step 2.4: Compile the CNN model
cnn_model.compile(optimizer='adam',
                  loss='binary_crossentropy', # Appropriate for binary classification
                  metrics=['accuracy'])

print("\nCNN Model Summary:")
cnn_model.summary()

# 🔹 Step 2.5: Train the CNN model
print("\nTraining CNN model...")
history = cnn_model.fit(X_train_cnn, y_train_balanced,
                        epochs=10, # Number of training epochs
                        batch_size=32, # Number of samples per gradient update
                        validation_split=0.2, # Use 20% of training data for validation
                        verbose=1) # Show training progress
print("CNN model training complete.")

# 🔹 Step 2.6: Make predictions
# Get probabilities, then convert to binary predictions (0 or 1)
y_pred_proba_cnn = cnn_model.predict(X_test_cnn)
y_pred_cnn = (y_pred_proba_cnn > 0.5).astype(int) # Threshold at 0.5

# 🔹 Step 2.7: Evaluation
print("\n" + "=" * 60)
print("             CNN MODEL EVALUATION             ")
print("=" * 60)
print("📊 Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_cnn))
print("\n📈 Classification Report:")
# Modified to display 4 decimal places for more precise values
print(classification_report(y_test, y_pred_cnn, digits=4))
print(f"🎯 Accuracy Score: {accuracy_score(y_test, y_pred_cnn):.4f}")
print("=" * 60 + "\n")


Reshaped X_train for CNN: (152108, 73, 1)
Reshaped X_test for CNN: (44617, 73, 1)

CNN Model Summary:


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_14 (Conv1D)              │ (None, 71, 32)         │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_14 (MaxPooling1D) │ (None, 35, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_39 (Dropout)            │ (None, 35, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_15 (Conv1D)              │ (None, 33, 64)         │         6,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_15 (MaxPooling1D) │ (None, 16, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_40 (Dropout)            │ (None, 16, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_7 (Flatten)             │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_38 (Dense)                │ (None, 128)            │       131,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_41 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_39 (Dense)                │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 137,665 (537.75 KB)

 Trainable params: 137,665 (537.75 KB)

 Non-trainable params: 0 (0.00 B)


Training CNN model...
Epoch 1/10
3803/3803 ━━━━━━━━━━━━━━━━━━━━ 51s 13ms/step - accuracy: 0.9822 - loss: 0.0587 - val_accuracy: 0.9991 - val_loss: 0.0059
Epoch 2/10
3803/3803 ━━━━━━━━━━━━━━━━━━━━ 81s 12ms/step - accuracy: 0.9984 - loss: 0.0106 - val_accuracy: 0.9985 - val_loss: 0.0059
Epoch 3/10
3803/3803 ━━━━━━━━━━━━━━━━━━━━ 86s 13ms/step - accuracy: 0.9986 - loss: 0.0075 - val_accuracy: 0.9990 - val_loss: 0.0056
Epoch 4/10
3803/3803 ━━━━━━━━━━━━━━━━━━━━ 79s 13ms/step - accuracy: 0.9990 - loss: 0.0053 - val_accuracy: 0.9993 - val_loss: 0.0039
Epoch 5/10
3803/3803 ━━━━━━━━━━━━━━━━━━━━ 83s 13ms/step - accuracy: 0.9990 - loss: 0.0047 - val_accuracy: 0.9990 - val_loss: 0.0048
Epoch 6/10
3803/3803 ━━━━━━━━━━━━━━━━━━━━ 47s 12ms/step - accuracy: 0.9990 - loss: 0.0049 - val_accuracy: 0.9992 - val_loss: 0.0032
Epoch 7/10
3803/3803 ━━━━━━━━━━━━━━━━━━━━ 87s 14ms/step - accuracy: 0.9990 - loss: 0.0052 - val_accuracy: 0.9991 - val_loss: 0.0027
Epoch 8/10
3803/3803 ━━━━━━━━━━━━━━━━━━━━ 75s 12ms/st

CNN model training complete.
CNN model saved as 'cnn_model.h5'
1395/1395 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step

             CNN MODEL EVALUATION             
📊 Confusion Matrix:
[[19007     7]
 [   22 25581]]

📈 Classification Report:
              precision    recall  f1-score   support

           0     0.9988    0.9996    0.9992     19014
           1     0.9997    0.9991    0.9994     25603

    accuracy                         0.9994     44617
   macro avg     0.9993    0.9994    0.9993     44617
weighted avg     0.9994    0.9994    0.9994     44617

🎯 Accuracy Score: 0.9994



VISUALIZE TRAINING HISTORY

In [43]:
import matplotlib.pyplot as plt

# Create subplots (1 row, 2 columns)
fig, ax = plt.subplots(1, 2, figsize=(14, 6))

# Plot training & validation accuracy
ax[0].plot(history.history['accuracy'])
ax[0].plot(history.history['val_accuracy'])
ax[0].set_title('Model Accuracy')
ax[0].set_ylabel('Accuracy')
ax[0].set_xlabel('Epoch')
ax[0].legend(['Train', 'Validation'], loc='upper left')

# Plot training & validation loss
ax[1].plot(history.history['loss'])
ax[1].plot(history.history['val_loss'])
ax[1].set_title('Model Loss')
ax[1].set_ylabel('Loss')
ax[1].set_xlabel('Epoch')
ax[1].legend(['Train', 'Validation'], loc='upper left')

# Adjust layout for better spacing
plt.tight_layout()
plt.show()


=================================================================================================================

ERIC TAN XIN ZHEN (ANN)

In [44]:
# This cell contains the code from ann.txt to define and train the ANN model.
# It uses the preprocessed data (X_train_scaled, y_train_balanced, X_test_scaled, y_test)
# that was generated by running the notebook from the GitHub URL.

# Code from ann.txt starts here:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd

# Define the ANN model
# The input dimension should match the number of features in your scaled training data (X_train_scaled)
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid') # Sigmoid for binary classification
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy', # Binary crossentropy for binary classification
              metrics=['accuracy'])

# Train the model
# Use the scaled and balanced training data
history = model.fit(X_train_scaled, y_train_balanced, epochs=10, batch_size=32, validation_split=0.2, verbose=1)

# Evaluate the model on the test data
# Use the scaled test data
loss, accuracy = model.evaluate(X_test_scaled, y_test, verbose=0)
print(f"\nTest Accuracy: {accuracy:.4f}") # Adjusted formatting to 4 decimal places

# Make predictions
y_pred_prob = model.predict(X_test_scaled)
y_pred = (y_pred_prob > 0.5).astype("int")

# Display classification report and confusion matrix
print("\n===================================")
print("\n--- Model Performance Summary ---")
print("\n===================================")
print(f"🎯Final Validation Accuracy: {history.history['val_accuracy'][-1]:.4f}") # Print final validation accuracy
print("\n📈Classification Report:")
print(classification_report(y_test, y_pred, digits=4)) # Adjusted formatting to 4 decimal places

print("\n📊Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Create a DataFrame from the training history
history_df = pd.DataFrame(history.history)

# Add an epoch column for easier readability
history_df['epoch'] = history_df.index + 1

# Display the DataFrame
print("\nTraining History Table:")
display(history_df[['epoch', 'accuracy', 'val_accuracy', 'loss', 'val_loss']])

# Code from ann.txt ends here.

Epoch 1/10


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


3803/3803 ━━━━━━━━━━━━━━━━━━━━ 17s 4ms/step - accuracy: 0.9857 - loss: 0.0455 - val_accuracy: 0.9991 - val_loss: 0.0041
Epoch 2/10
3803/3803 ━━━━━━━━━━━━━━━━━━━━ 14s 4ms/step - accuracy: 0.9990 - loss: 0.0047 - val_accuracy: 0.9993 - val_loss: 0.0038
Epoch 3/10
3803/3803 ━━━━━━━━━━━━━━━━━━━━ 20s 4ms/step - accuracy: 0.9990 - loss: 0.0052 - val_accuracy: 0.9994 - val_loss: 0.0039
Epoch 4/10
3803/3803 ━━━━━━━━━━━━━━━━━━━━ 14s 4ms/step - accuracy: 0.9991 - loss: 0.0040 - val_accuracy: 0.9994 - val_loss: 0.0029
Epoch 5/10
3803/3803 ━━━━━━━━━━━━━━━━━━━━ 15s 4ms/step - accuracy: 0.9992 - loss: 0.0047 - val_accuracy: 0.9993 - val_loss: 0.0034
Epoch 6/10
3803/3803 ━━━━━━━━━━━━━━━━━━━━ 20s 4ms/step - accuracy: 0.9993 - loss: 0.0037 - val_accuracy: 0.9991 - val_loss: 0.0036
Epoch 7/10
3803/3803 ━━━━━━━━━━━━━━━━━━━━ 21s 4ms/step - accuracy: 0.9992 - loss: 0.0032 - val_accuracy: 0.9993 - val_loss: 0.0028
Epoch 8/10
3803/3803 ━━━━━━━━━━━━━━━━━━━━ 15s 4ms/step - accuracy: 0.9994 - loss: 0.0033 - val

,epoch,accuracy,val_accuracy,loss,val_loss
0,1,0.995899,0.999145,0.017335,0.004080
1,2,0.998792,0.999343,0.005143,0.003838
2,3,0.999063,0.999375,0.004721,0.003939
3,4,0.999129,0.999375,0.004099,0.002927
4,5,0.999203,0.999277,0.004237,0.003378
5,6,0.999236,0.999112,0.003628,0.003583
6,7,0.999301,0.999343,0.003291,0.002849
7,8,0.999285,0.999112,0.003427,0.004677
8,9,0.999293,0.999408,0.003213,0.003240
9,10,0.999252,0.999244,0.003910,0.002460


In [45]:
import matplotlib.pyplot as plt

# Create subplots (1 row, 2 columns)
fig, ax = plt.subplots(1, 2, figsize=(14, 6))

# Plot training and validation accuracy
ax[0].plot(history.history['accuracy'], label='Training Accuracy')
ax[0].plot(history.history['val_accuracy'], label='Validation Accuracy')
ax[0].set_title('Model Accuracy')
ax[0].set_xlabel('Epoch')
ax[0].set_ylabel('Accuracy')
ax[0].legend()
ax[0].grid(True)

# Plot training and validation loss
ax[1].plot(history.history['loss'], label='Training Loss')
ax[1].plot(history.history['val_loss'], label='Validation Loss')
ax[1].set_title('Model Loss')
ax[1].set_xlabel('Epoch')
ax[1].set_ylabel('Loss')
ax[1].legend()
ax[1].grid(True)

# Adjust spacing
plt.tight_layout()
plt.show()


=================================================================================================================

AMOS WAI ZHAN (Decision Tree)

In [46]:
# ➕ Decision Tree Model

# Import necessary libraries
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, RocCurveDisplay
import matplotlib.pyplot as plt

# Replace these with your actual variable names if different
# X_train, X_test, y_train, y_test must already be defined from CleanedData.ipynb

# Convert labels to integers (e.g., from strings like '0' or '1')
y_train = y_train.astype(str).str.strip()  # Clean whitespace
y_train = y_train.astype(int)

y_test = y_test.astype(str).str.strip()
y_test = y_test.astype(int)


print("🌲 Training Decision Tree Classifier...")
dt_model = DecisionTreeClassifier(random_state=42)
dt_model.fit(X_train, y_train)

print("🔍 Evaluating the model...")
y_pred = dt_model.predict(X_test)
y_prob = dt_model.predict_proba(X_test)[:, 1]

# Metrics
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_prob)

print(f"✅ Accuracy: {acc:.4f}")
print(f"✅ Precision: {prec:.4f}")
print(f"✅ Recall: {rec:.4f}")
print(f"✅ F1 Score: {f1:.4f}")
print(f"✅ ROC-AUC: {roc_auc:.4f}")

# Confusion Matrix
print("\n📊 Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Plot ROC Curve
RocCurveDisplay.from_estimator(dt_model, X_test, y_test)
plt.title("ROC Curve - Decision Tree")
plt.grid(True)
plt.show()

# Final verdict
print(f"\n🎯 Verdict: Achieved F1 = {f1:.2f}, {'target met ✅' if f1 >= 0.90 else 'target not met ❌'}")

🌲 Training Decision Tree Classifier...
🔍 Evaluating the model...
✅ Accuracy: 0.9998
✅ Precision: 0.9998
✅ Recall: 0.9999
✅ F1 Score: 0.9998
✅ ROC-AUC: 0.9998

📊 Confusion Matrix:
[[19009     5]
 [    3 25600]]

🎯 Verdict: Achieved F1 = 1.00, target met ✅


=================================================================================================================

GUI

In [90]:
# streamlit_app.py  — CNN / ANN / DecisionTree
import streamlit as st, joblib, numpy as np, pandas as pd, io, os
import tensorflow as tf

st.set_page_config(page_title="Single Prediction (CNN / ANN / DT)")
st.title("Single Prediction — CNN / ANN / Decision Tree")

# ==== MODEL PATHS: update these paths if your filenames are different ====
CNN_PATH = "cnn_model.h5"
ANN_PATH = "ann_model.h5"
DT_PATH  = "decision_tree_model.pkl"
# ============================================================================

def safe_load(path_label, path_value, model_type):
    try:
        if path_value and os.path.exists(path_value):
            if model_type == "keras":
                return tf.keras.models.load_model(path_value)
            elif model_type == "joblib":
                return joblib.load(path_value)
    except Exception as e:
        st.warning(f"Could not load {path_label} from path: {e}")
    return None

models = {}
# Load models using the correct types and paths
cnn = safe_load("CNN", CNN_PATH, "keras")
ann = safe_load("ANN", ANN_PATH, "keras")
dt  = safe_load("Decision Tree", DT_PATH, "joblib")


if cnn: models["CNN"] = cnn
if ann: models["ANN"] = ann
if dt:  models["Decision Tree"] = dt


if not models:
    st.error("No models loaded. Ensure model files are in the correct location and named as specified.")
    st.stop()

st.success("Loaded: " + ", ".join(models.keys()))
model_name = st.selectbox("Choose model", list(models.keys()))
pipe = models[model_name]

mode = st.radio("Input", ["Paste text/URL", "Upload .txt"])
if mode == "Paste text/URL":
    text = st.text_area("Sample:", height=120, placeholder="Paste one sample…")
else:
    up_txt = st.file_uploader("Upload a .txt file", type=["txt"], key="txt_up")
    text = up_txt.read().decode("utf-8", "ignore") if up_txt else ""

if st.button("Predict"):
    if not (text or "").strip():
        st.warning("Please provide input.")
    else:
        try:
            # Need to preprocess the input data to match the model's training data
            # This part requires the original data preprocessing steps to be available
            # For now, assuming a placeholder preprocessing step
            # This will need to be replaced with the actual data preprocessing logic
            # before feeding the input to the loaded model for accurate predictions.
            # Example placeholder:
            # processed_input = preprocess_input(text) # This function needs to be defined

            # Dummy data for demonstration purposes (replace with actual processed input)
            # The shape should match the input shape the model expects (number of features)
            # You'll need to get the number of features from your training data
            # For the CNN, it should be (1, num_features, 1)
            # For ANN and DT, it should be (1, num_features)
            # Example: Assuming 73 features from the scaling step
            num_features = 73 # Replace with the actual number of features

            # For CNN
            if model_name == "CNN":
                # Dummy input for CNN (batch_size=1, num_features, 1)
                dummy_input = np.random.rand(1, num_features, 1)
                proba = pipe.predict(dummy_input)[0]
            # For ANN and Decision Tree
            else:
                 # Dummy input for ANN/DT (batch_size=1, num_features)
                 dummy_input = np.random.rand(1, num_features)
                 # Decision Tree might not have predict_proba if trained differently,
                 # so use a try-except block or check model type.
                 if hasattr(pipe, 'predict_proba'):
                     proba = pipe.predict_proba(dummy_input)[0]
                 else:
                      # If no predict_proba, use predict and create a probability array
                      pred_class = pipe.predict(dummy_input)[0]
                      # Assuming binary classification (0 or 1)
                      proba = np.array([0.0, 0.0])
                      if pred_class == 0:
                          proba[0] = 1.0
                      else:
                          proba[1] = 1.0

            # Define classes - this should align with how your models were trained
            # Assuming 0 for BENIGN and 1 for DDoS based on previous output
            classes = np.array(["BENIGN", "DDoS"])


        except Exception as e:
            st.error(f"Prediction error: {e}")
            proba = None # Indicate prediction failed

        if proba is not None:
            # Assuming binary classification with two probabilities
            if len(proba) == 2:
                pred_index = np.argmax(proba)
                pred = classes[pred_index]
                st.subheader(f"Prediction: **{pred}**")
                # Display probabilities for each class
                proba_dict = {classes[i]: float(proba[i]) for i in range(len(classes))}
                st.write(proba_dict)
                st.bar_chart(pd.DataFrame({"probability": proba}, index=classes))
            else:
                # Handle cases where model output is not binary probabilities
                 st.subheader(f"Prediction Output:")
                 st.write(proba) # Display raw model output

2025-08-22 17:21:57.026 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-22 17:21:57.027 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-22 17:21:57.028 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-22 17:21:57.029 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-22 17:21:57.600 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-22 17:21:57.605 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-22 17:21:57.606 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-22 17:21:57.615 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [96]:
import gradio as gr
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler # Import StandardScaler

# Assume models (cnn, ann, dt) and scaler (scaler) are already loaded from previous steps
# If not, you'll need to load them here.
# Assuming 'scaler' object is available from the preprocessing steps

# Define the original feature column names based on your notebook output
# Ensure this list matches the order of columns in your X_train before feature engineering
original_feature_names = [
    'Destination Port', 'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets',
    'Total Length of Fwd Packets', 'Total Length of Bwd Packets', 'Fwd Packet Length Max',
    'Fwd Packet Length Min', 'Fwd Packet Length Mean', 'Fwd Packet Length Std',
    'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Mean',
    'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean',
    'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean',
    'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean',
    'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Fwd Header Length',
    'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s', 'Min Packet Length',
    'Max Packet Length', 'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance',
    'FIN Flag Count', 'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count',
    'URG Flag Count', 'ECE Flag Count', 'Down/Up Ratio', 'Average Packet Size',
    'Avg Fwd Segment Size', 'Avg Bwd Segment Size', 'Fwd Header Length.1',
    'Subflow Fwd Packets', 'Subflow Fwd Bytes', 'Subflow Bwd Packets', 'Subflow Bwd Bytes',
    'Init_Win_bytes_forward', 'Init_Win_bytes_backward', 'act_data_pkt_fwd',
    'min_seg_size_forward', 'Active Mean', 'Active Std', 'Active Max', 'Active Min',
    'Idle Mean', 'Idle Std', 'Idle Max', 'Idle Min'
]

def preprocess_input(input_data_str, scaler=None):
    """
    Preprocesses the raw input data string into numerical features.

    Args:
        input_data_str (str): The raw input data as a string (comma-separated values).
        scaler (StandardScaler, optional): The fitted scaler object. Defaults to None.

    Returns:
        tuple: A tuple containing:
            - processed_input_original (np.ndarray): Original 68 features.
            - processed_input_engineered (np.ndarray): 73 features after engineering.
            - processed_input_scaled (np.ndarray): Scaled engineered features (or None if scaler is None).
            - error (str or None): Error message if preprocessing fails.
    """
    original_num_features = len(original_feature_names)
    input_values = []
    error = None

    input_values_str = [x.strip() for x in input_data_str.split(',')]

    if len(input_values_str) != original_num_features:
         error = f"Error: Expected {original_num_features} comma-separated feature values, but got {len(input_values_str)}."
         return None, None, None, error

    for val_str in input_values_str:
        try:
            input_values.append(float(val_str))
        except ValueError:
            error = f"Error: Could not convert value '{val_str}' to a number. Please ensure all inputs are numerical."
            return None, None, None, error

    processed_input_original = np.array(input_values).reshape(1, -1)
    processed_input_original_df = pd.DataFrame(processed_input_original, columns=original_feature_names)

    # Replicate Feature Engineering steps using column names
    try:
        pkt_length_diff = processed_input_original_df['Max Packet Length'] - processed_input_original_df['Min Packet Length']
        # Handle division by zero
        with np.errstate(divide='ignore', invalid='ignore'):
            pkt_length_var_ratio = np.where(processed_input_original_df['Packet Length Mean'] != 0, processed_input_original_df['Packet Length Variance'] / processed_input_original_df['Packet Length Mean'], 0)
            byte_ratio = np.where(processed_input_original_df['Total Length of Bwd Packets'] != 0, processed_input_original_df['Total Length of Fwd Packets'] / processed_input_original_df['Total Length of Bwd Packets'], 0)
            duration_per_packet = np.where((processed_input_original_df['Total Fwd Packets'] + processed_input_original_df['Total Backward Packets']) != 0, processed_input_original_df['Flow Duration'] / (processed_input_original_df['Total Fwd Packets'] + processed_input_original_df['Total Backward Packets']), 0)
            avg_to_max_ratio = np.where(processed_input_original_df['Max Packet Length'] != 0, processed_input_original_df['Average Packet Size'] / processed_input_original_df['Max Packet Length'], 0)


        # Combine original and new features
        processed_input_engineered = np.hstack([
            processed_input_original,
            pkt_length_diff.values.reshape(-1, 1),
            pkt_length_var_ratio.reshape(-1, 1),
            byte_ratio.reshape(-1, 1),
            duration_per_packet.reshape(-1, 1),
            avg_to_max_ratio.reshape(-1, 1)
        ])

    except Exception as e:
        error = f"Error during feature engineering: {e}"
        return None, None, None, error

    processed_input_scaled = None
    if scaler is not None:
        try:
            processed_input_scaled = scaler.transform(processed_input_engineered)
        except Exception as e:
            error = f"Error during scaling: {e}"
            return None, None, None, error


    return processed_input_original, processed_input_engineered, processed_input_scaled, None


def predict(input_data, model_choice):
    """
    Predicts the class of the input data using the chosen model.

    Args:
        input_data (str): The raw input data as a string (e.g., comma-separated values).
        model_choice (str): The name of the model to use ('CNN', 'ANN', 'Decision Tree').

    Returns:
        str: The predicted class ('BENIGN' or 'DDoS') and potentially probabilities.
    """
    # Assume models (cnn, ann, dt) and scaler (scaler) are loaded globally

    processed_input_original, processed_input_engineered, processed_input_scaled, error = preprocess_input(input_data, scaler)

    if error:
        return error

    # 2. Make prediction using the chosen model
    try:
        if model_choice == 'CNN':
            if 'cnn' not in globals() or cnn is None:
                 return "Error: CNN model not loaded."
            # Reshape for CNN (samples, timesteps/features, channels)
            processed_input_for_prediction = processed_input_scaled.reshape(processed_input_scaled.shape[0], processed_input_scaled.shape[1], 1)
            proba = cnn.predict(processed_input_for_prediction)[0]
            prediction = (proba > 0.5).astype(int)
            classes = ["BENIGN", "DDoS"] # Assuming 0: BENIGN, 1: DDoS
            predicted_class = classes[prediction[0]]
            output_text = f"Prediction: {predicted_class} | Probabilities: {classes[prediction[0]]}: {proba[0]:.4f}, {classes[1-prediction[0]]}: {1-proba[0]:.4f}"


        elif model_choice == 'ANN':
            if 'ann' not in globals() or ann is None:
                 return "Error: ANN model not loaded."
            processed_input_for_prediction = processed_input_scaled # ANN expects scaled engineered features
            proba = ann.predict(processed_input_for_prediction)[0]
            prediction = (proba > 0.5).astype(int)
            classes = ["BENIGN", "DDoS"] # Assuming 0: BENIGN, 1: DDoS
            predicted_class = classes[prediction[0]]
            output_text = f"Prediction: {predicted_class} | Probabilities: {classes[prediction[0]]}: {proba[0]:.4f}, {classes[1-prediction[0]]}: {1-proba[0]:.4f}"


        elif model_choice == 'Decision Tree':
            if 'dt' not in globals() or dt is None:
                 return "Error: Decision Tree model not loaded."
            processed_input_for_prediction = processed_input_original # DT expects original features
            prediction = dt.predict(processed_input_for_prediction)[0]
            if hasattr(dt, 'predict_proba'):
                proba = dt.predict_proba(processed_input_for_prediction)[0]
                classes = ["BENIGN", "DDoS"] # Assuming 0: BENIGN, 1: DDoS
                predicted_class = classes[prediction]
                output_text = f"Prediction: {predicted_class} | Probabilities: " + ", ".join([f"{classes[i]}: {p:.4f}" for i, p in enumerate(proba)])
            else:
                classes = ["BENIGN", "DDoS"] # Assuming 0: BENIGN, 1: DDoS
                predicted_class = classes[prediction]
                output_text = f"Prediction: {predicted_class} (Probability information not available)"

        else:
            output_text = "Error: Invalid model choice."

    except Exception as e:
        output_text = f"Error during prediction: {e}"

    return output_text

# Placeholder for loading models - replace with your actual loading code if needed
# Assuming models (cnn, ann, dt) and scaler are already loaded in the environment.
# If not, you would load them here:
# cnn = tf.keras.models.load_model('cnn_model.h5')
# ann = tf.keras.models.load_model('ann_model.h5') # Assuming ANN is also Keras
# dt = joblib.load('decision_tree_model.pkl')
# scaler = joblib.load('scaler.pkl') # Load your scaler if used

print("Predict function defined. Ready to create Gradio interface.")

Predict function defined. Ready to create Gradio interface.


In [97]:
# Assuming 'predict' function, input_components, and output_component are defined

# Create the Gradio Interface
iface = gr.Interface(
    fn=predict,
    inputs=input_components,
    outputs=output_component,
    title="DDoS Attack Prediction",
    description="Enter 73 comma-separated feature values and select a model to predict if it's a DDoS attack or BENIGN."
)

print("Gradio interface created.")

Gradio interface created.


In [98]:
# Assuming 'predict' function and models (cnn, ann, dt) are defined and loaded

# Define Gradio input components
input_components = [
    gr.Textbox(label="Enter comma-separated feature values (73 values)"),
    gr.Dropdown(choices=["CNN", "ANN", "Decision Tree"], label="Select Model")
]

# Define Gradio output component
output_component = gr.Textbox(label="Prediction Result")

print("Gradio input and output components defined.")

Gradio input and output components defined.


In [99]:
# Assuming 'iface' is defined

# Launch the Gradio interface
iface.launch(debug=True)

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://70da907edc33965643.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7871 <> https://70da907edc33965643.gradio.live
